In [1]:
import random
import pandas as pd
import numpy as np

# установка seed
random.seed("АДВИ_2023")

# генерация данных
N = 1000

index = list(range(1, N+1))
random.shuffle(index)

groups = random.choices(["группа_1", "группа_2", "группа_3", "группа_4", "группа_5"], k=N)

uniform = [random.randint(50, 100) for i in range(N)]

gauss_1 = [random.gauss(0, 1) for i in range(N)]
gauss_2 = [random.gauss(0, 12) for i in range(N)]
gauss_3 = [random.gauss(50, 9) for i in range(N)]

# создание DataFrame
data = {"index": index, "groups": groups, "uniform": uniform, "gauss_1": gauss_1, "gauss_2": gauss_2, "gauss_3": gauss_3}
df = pd.DataFrame(data)
df = df.set_index("index")

# замена значений на None
df.loc[df.index % 121 == 0, "gauss_1"] = None
df.loc[np.abs(df["gauss_2"] % 1 - 0.95) < 0.0001, "gauss_2"] = None

# замена пропущенных значений
gauss_1_mean = df["gauss_1"].mean()
df["gauss_1"].fillna(gauss_1_mean, inplace=True)

# удаление строк с None значениями в gauss_1
df = df[df["gauss_1"].notna()]

print(df.to_string())

         groups  uniform   gauss_1    gauss_2    gauss_3
index                                                   
476    группа_1       95  3.010291   2.156346  57.455851
315    группа_5       77  0.190277  -9.934550  62.694177
490    группа_4       96 -0.282161   9.211071  54.688895
748    группа_3       56  1.127080 -12.391793  30.192353
471    группа_5       52 -1.238270  10.938748  52.682052
907    группа_2       70 -1.227963  12.721383  55.538879
457    группа_2       70 -0.553928  -1.674862  48.011108
826    группа_1       73  0.581783  -2.170896  49.992722
5      группа_3       64  1.612765  -1.242515  41.876322
646    группа_3       86 -0.485014  10.381790  55.025045
477    группа_5       61  0.153584  -1.530909  50.335015
392    группа_4       96 -0.022030 -15.327097  47.695130
309    группа_4       62  0.690958 -13.272665  32.448785
136    группа_4       86 -1.033991  -3.833145  67.267556
655    группа_2      100  0.148248   9.559408  49.953550
816    группа_4       89 -1.303

In [3]:
gauss_2_mean = df['gauss_2'].mean().round(2)
gauss_2_std = df['gauss_2'].std().round(2)
result = f"{gauss_2_mean} {gauss_2_std}"
print(result)


-0.16 12.23


In [4]:
group_counts = df['groups'].value_counts()
most_frequent_group = group_counts.index[0]
count_most_frequent_group = group_counts[most_frequent_group]
print(f"{most_frequent_group} {count_most_frequent_group}")


группа_4 225


**Для группа_5 вывести самый "ранний" (наименьший) индекс.**

In [6]:
index_min = df.loc[df['groups'] == 'группа_5'].index.min()
print(index_min)

6


**Сколько записей из колонки uniform имеют значение не ниже значения 90% перцентиля?**

In [8]:
pct90 = df["uniform"].quantile(q=0.9)
above_pct90 = df[df["uniform"] >= pct90]
count_above_pct90 = above_pct90["uniform"].count()
print(count_above_pct90)

119


**У какой группы наибольшее значение медианы для gauss_3?**

* Вывести это значение вместе с группой. Сначала группа, через пробел значение медианы (результат округлить до сотых , для дробной части использовать точку, как разделитель)

In [9]:
result = df.groupby("groups")["gauss_3"].median()
max_median = result.max()
group_with_max_median = result[result == max_median].index[0]
print(f"{group_with_max_median} {max_median:.2f}")

группа_5 50.00


**У какой группы наибольшее минимальное значение uniform?**

* Если такого нет, выбрать группу наибольшим минимальным значением в gauss_1.

* Ответ дать в виде группы, через пробел указать значение в gauss_2 с аналогичной агрегацией (результат округлить до сотых, для дробной части использовать точку, как разделитель)

In [10]:
grouped = df.groupby("groups")
min_uniform_group = grouped.apply(lambda x: x.loc[x["uniform"].idxmin()]["groups"]).iloc[0]
if df.loc[df["groups"] == min_uniform_group]["uniform"].min() >= df["uniform"].min():
    max_gauss_1_group = grouped.apply(lambda x: x.loc[x["gauss_1"].idxmin()]["groups"]).value_counts().idxmax()
    max_gauss_1_value = df.loc[df["groups"] == max_gauss_1_group]["gauss_2"].min()
    result = f"{max_gauss_1_group} {max_gauss_1_value:.2f}"
else:
    max_uniform_value = df.loc[df["groups"] == min_uniform_group]["gauss_2"].max()
    result = f"{min_uniform_group} {max_uniform_value:.2f}"
    
print(result)

группа_1 -22.11


**Создать новую колонку gauss где значения будут получаться из (gauss_1 + gauss_2) / gauss_3**

**Чему будет равен 70% перцентиль (результат округлить до сотых, для дробной части использовать точку, как разделитель)?**

In [12]:
df["gauss"] = (df["gauss_1"] + df["gauss_2"]) / df["gauss_3"]
percentile_70 = df["gauss"].quantile(q=0.7).round(2)
print(percentile_70)

0.12
